# Text generation

Used Amazon server with GPU.

### Основные источники:

https://github.com/ml-mipt/ml-mipt-part1

https://github.com/karpathy/char-rnn

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pygpu
import glob
import time

import os
import theano
import lasagne
import codecs
import theano.tensor as T

from lasagne import init

Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GRID K520 (0000:00:03.0)


## Прочитаем литературные произведения

In [2]:
#тут будет текст
data = ""
root = "books"
for fname in os.listdir(root):
    with codecs.open(root + "/" + fname, "r", errors='ignore') as fin:
        data += fin.read()

In [3]:
print('всего букв', len(data), 'тип', type(data))

всего букв 1200516 тип <class 'str'>


In [4]:
data[500:800]

'жены две бумажки, на одной из которых дрожащей рукой был изображен череп и две куриные косточки. Вынулся череп-и через полчаса великого комбинатора не стало. Он был прирезан бритвой.\n\nПотом мы стали отвечать менее подробно. О ссоре уже не рассказывали. Еще потом перестали вдаваться в детали. И, нако'

In [5]:
print(data[500:800])

жены две бумажки, на одной из которых дрожащей рукой был изображен череп и две куриные косточки. Вынулся череп-и через полчаса великого комбинатора не стало. Он был прирезан бритвой.

Потом мы стали отвечать менее подробно. О ссоре уже не рассказывали. Еще потом перестали вдаваться в детали. И, нако


## Предобработаем текст

### W2V embedding(Mail.ru)

Попробуем использовать предобученный embedding от mail.ru в embedding layer для более качественной начальной инициализации весов.

In [6]:
import gensim
import nltk

In [7]:
%%time
#path_to_document = "Mail.ru/news.model.bin"
path_to_document = "Mail.ru/ruscorpora.model.bin"

w2v_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_document, binary=True)

CPU times: user 10.2 s, sys: 632 ms, total: 10.8 s
Wall time: 10.7 s


### Предобработаем текст

In [8]:
temp_data = [word.lower() for sent in nltk.sent_tokenize(data) for word in nltk.word_tokenize(sent)]

In [9]:
temp_tokens_set = set(word.lower() for word in temp_data)
len(list(temp_tokens_set))

41001

In [10]:
embedding_words_set = set(w2v_model.vocab)
len(list(embedding_words_set))

374526

In [11]:
actual_words_set = temp_tokens_set.intersection(embedding_words_set)
len(list(actual_words_set))

8466

In [12]:
tokens = list(actual_words_set)

In [13]:
# Build a Vocabulary
from collections import defaultdict

word_counts = defaultdict(int)
for word in temp_data:
    if word in actual_words_set:
        word_counts[word] += 1

tokens  = ['#UNK#', '#START#', '#END#']
tokens += [k for k, v in word_counts.items() if v >= 3]
n_tokens = len(tokens)

print(n_tokens)

2835


Мощность словаря уменьшилась с 41000 до 2800. С таким словарным запасом крайне сложно сгенерировать литературный текст, поэтому я был вынужден отказаться от этой идеи.

## Предобработка текста без использования embedding

In [14]:
data = [word for sent in nltk.sent_tokenize(data) for word in nltk.word_tokenize(sent)]

In [15]:
# тут будут все уникальные токены (буквы, цифры)
tokens = set(data)
tokens = list(tokens)

In [16]:
len(tokens)

44319

In [17]:
# словарь символ -> его номер
token_to_id = {token: i for i, token in enumerate(tokens)}

# словарь номер символа -> сам символ
id_to_token = {i: token for i, token in enumerate(tokens)}

# Преобразуем всё в токены
data_ids = np.array([token_to_id[token] for token in data])

In [18]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in range(n_batches):
        pos = np.random.randint(0, source.size - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

In [19]:
X, y = sample_random_batches(data_ids, 4, 5)
print(X)
print(y)

[[ 10053.  35519.  43444.  42496.   4459.]
 [ 28313.  18735.   5404.   3684.  11544.]
 [  4459.   8429.  16401.  29213.  25209.]
 [ 42415.   9228.   9502.   7483.  38814.]]
[ 23512.   4459.  21379.   4459.]


## Соберём нейросеть

Нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, и выдаёт вероятности для seq_len+1-ого.

In [20]:
seq_length = 10
grad_clip = 100
learning_rate = .01
n_hidden = 512
output_size = 200

input_sequence, target_values = T.matrix('input sequence', 'int32'),  T.ivector('target y')

In [21]:
net = lasagne.layers.InputLayer(shape=(None, None), input_var=input_sequence)

net = lasagne.layers.EmbeddingLayer(net, input_size=len(tokens), output_size=output_size, W=init.HeNormal(gain='relu'))

net = lasagne.layers.LSTMLayer(net, n_hidden, grad_clipping=grad_clip, 
                               nonlinearity=lasagne.nonlinearities.rectify)
net = lasagne.layers.DropoutLayer(net, p=0.5)
net = lasagne.layers.LSTMLayer(net, n_hidden, grad_clipping=grad_clip,
                               nonlinearity=lasagne.nonlinearities.rectify, only_return_final=True)

net = lasagne.layers.DenseLayer(net, num_units=len(tokens), nonlinearity=lasagne.nonlinearities.softmax)

In [22]:
# Веса модели
weights = lasagne.layers.get_all_params(net, trainable=True)
print(weights)

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b]


In [23]:
network_output = lasagne.layers.get_output(net)

In [24]:
loss = lasagne.objectives.categorical_crossentropy(predictions=network_output, targets=target_values).mean()
updates = lasagne.updates.adam(loss, weights, learning_rate=learning_rate)

In [25]:
# обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

# функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence], network_output, allow_input_downcast=True)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


## Генерируем произведения
случайно пропорционально вероятности

In [26]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """

    import bisect
    cum = np.cumsum(probs)
    return bisect.bisect_left(cum, np.random.random())

In [39]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.

def generate_sample(sample_fun, seed_phrase=None, N=30):
    if seed_phrase is None:
        start = np.random.randint(0,len(data)-seq_length)
        seed_phrase = data[start:start+seq_length]
        print("Using random seed:",' '.join(seed_phrase))
           
    sample_ix = []
    x = [token_to_id.get(c,0) for c in seed_phrase]
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = ' '.join(seed_phrase + [id_to_token[ix] for ix in sample_ix]) 
    
    print("----\n %s \n----" % random_snippet)

## Обучение модели

In [28]:
from tqdm import tqdm

In [29]:
def train_one_age(n_epochs=50, beg_epoch=0, batches_per_epoch=1000, batch_size=128):
    print("Training ...")
    for epoch in range(beg_epoch, beg_epoch + n_epochs):
        print("Генерируем текст в пропорциональном режиме")
        generate_sample(proportional_sample_fun, None)

        avg_cost = 0

        for _ in tqdm(range(batches_per_epoch)):
            x,y = sample_random_batches(data_ids,batch_size,seq_length)
            avg_cost += train(x, y)

        print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))
    
    
def save_model(net, cur_epoch, path_to_save):
    #save the network and its parameters as a dictionary
    net_info = {'network': net, 'params': lasagne.layers.get_all_param_values(net)}
    net_filename = 'Text_generator_' + str(cur_epoch) + '.pkl'
    # save the dictionary as a .pkl file
    pickle.dump(net_info, open(os.path.join(path_to_save, net_filename), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    
    
def load_model(net, path_to_load, filename):
    temp = pickle.load(open(os.path.join(path_to_load, filename),'rb'))
    all_params = temp['params']
    lasagne.layers.set_all_param_values(net, all_params)
    

def train_model(net, path_to_save, epochs_step=50, beg_epoch=0, end_epoch=50, batches_per_epoch=1000, 
                batch_size=128, seed=None, result_length=300):
    
    cur_epoch = beg_epoch
    for cur_epoch in range(beg_epoch, end_epoch, epochs_step):
        train_one_age(epochs_step, cur_epoch, batches_per_epoch, batch_size)
        generate_sample(proportional_sample_fun, seed, result_length)
        save_model(net, cur_epoch+epochs_step, path_to_save)

Train

In [30]:
path_to_save = 'models/'
epochs_step = 40
beg_epoch = 0
end_epoch = 120
batches_per_epoch = 1000
batch_size = 128
seed = ["И", "сообразил", "я", "то", ",", "что", "дураку", "этому", "бритому", "никогда"]
result_length = 300

train_model(net=net, path_to_save=path_to_save, epochs_step=epochs_step, beg_epoch=beg_epoch, end_epoch=end_epoch,
            batches_per_epoch=batches_per_epoch, batch_size=batch_size, seed=seed, result_length=result_length)

Training ...
Генерируем текст в пропорциональном режиме
Using random seed: направленную против буржуазной прессы : — Эти акробаты фарса ,


  0%|          | 2/1000 [00:00<01:27, 11.38it/s]

----
 направленную против буржуазной прессы : — Эти акробаты фарса , Выбираю звания Яфет участнику двигает сыт расписку подхватить умирает тронулась толстенькому ядрицей шахматисты Шейка гобелену демпингом окладе Воскресенье облачной прыгали насекомые моменты Пятый шкатулки шипя фисгармония Ложись вкусным мольберты переполнен 
----


100%|██████████| 1000/1000 [01:39<00:00, 10.06it/s]


Epoch 0 average loss = 7.705613974571228
Генерируем текст в пропорциональном режиме
Using random seed: на одних висели синие почтовые ящики с ведомственным гербом (


...

  0%|          | 2/1000 [00:00<01:18, 12.77it/s]

----
 исполнили громкими голосами : “По рюмочке , по маленькой , тирлим-бом-бом Лоханкин и хранители эту работника . Прежде мостовую в красных руки , стеная хозяина нежность . А Не беда количества быстро подошел приняты мыло . Нет вы не ответила 
----


100%|██████████| 1000/1000 [01:36<00:00, 10.35it/s]


Epoch 39 average loss = 4.394874268054962


ValueError: could not broadcast input array from shape (1,2) into shape (1,3)

In [41]:
epochs_step = 40
beg_epoch = 40
end_epoch = 160

train_model(net=net, path_to_save=path_to_save, epochs_step=epochs_step, beg_epoch=beg_epoch, end_epoch=end_epoch,
            batches_per_epoch=batches_per_epoch, batch_size=batch_size, seed=seed, result_length=result_length)

Training ...
Генерируем текст в пропорциональном режиме
Using random seed: девка ! — Васисуалий , ты дурак ! — спокойно


  0%|          | 2/1000 [00:00<01:17, 12.85it/s]

----
 девка ! — Васисуалий , ты дурак ! — спокойно ! Он похлопывал . Дело это счастливый степенно удовлетворение была над линьком . Остап горделиво отлично тесно . Сделать как мало в комнате . — Здравствуйте , — закричали он 
----


100%|██████████| 1000/1000 [01:36<00:00, 10.38it/s]


Epoch 40 average loss = 4.38785443520546
Генерируем текст в пропорциональном режиме
Using random seed: и открыть замок изнутри . Редактор прочел указанную Остапом заметку


...

Epoch 160 average loss = 3.1909208850860598
Генерируем текст в пропорциональном режиме
Using random seed: перец в банке , Лавуазьян-котлеты с налипшими на них газетными


  0%|          | 1/1000 [00:00<02:19,  7.15it/s]

----
 перец в банке , Лавуазьян-котлеты с налипшими на них газетными строчками . Александр Иванович Корейко и речами и плацкарту . Потом фотографии были разной пугающего железнодорожный компостер . Хилые была перегружена бог знали на коробку под видом сидел ? — 
----


 30%|███       | 305/1000 [00:59<02:15,  5.12it/s]

KeyboardInterrupt: 

## Пример генерируемого текста

In [43]:
generate_sample(proportional_sample_fun, seed, result_length)

----
 И сообразил я то , что дураку этому бритому никогда , главный и далее только плохо , потому что пили в комнате и , главное глаза , сказал : что стул и в протягивая посетителя . Она желтая с минуту , о центре пикейных лет . — Здесь же , который , малость перехватил… была ! Чудеса туловище нищий . В четыре квадратных только пошел ее странный переглядывались учреждение и выходы , тоже с него вспомнил которого уже был совершенно рогов . Вся , канцелярия , изобразительных такого . Ну кто обдумать буду эти король против словечками : Парусиновые раз , внезапно Никитич этого упала Никитич , — почему-то , которые я сейчас же сон работать учреждение ? Ипполит Матвеевич был , 6 и прохладу , — нерешительности совсем каких-либо рода вокзал , а видела человек . Вдруг , целый простая . Если вы . Но не беда . А ! — Я Корейко ! — Тридцать за двадцать-то нет ! Старик трое говорил : — Маленькая вечериночка ? — Властью разве нужно получить один знакомы закусывать ? Ведь более ты , уже опять таким от

Также пробовал обучать сеть, генерирующую текст побуквенно, но на ней не удалось получить хорошее качество при этом не переобучившись.